In [1]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
import spacy 
from spacy import displacy
#import neptune.new as neptune                          
from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset, Dataset)

In [2]:
import torch

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla P100-PCIE-16GB, n_gpu: 1


In [3]:
!pip install transformers
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('success!')

import os
import zipfile

# Download helper functions file
helper_file = drive.CreateFile({'id': '16HW-z9Y1tM3gZ_vFpJAuwUDohz91Aac-'})
helper_file.GetContentFile('helpers.py')
print('helper file downloaded! (helpers.py)')

# Download sample file of tweets
data_file = drive.CreateFile({'id': '1QcoAmjOYRtsMX7njjQTYooIbJHPc6Ese'})
data_file.GetContentFile('tweets.csv')
print('sample tweets downloaded! (tweets.csv)')

success!
helper file downloaded! (helpers.py)
sample tweets downloaded! (tweets.csv)


In [4]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
path = 'nlp_data/pretrained_dir' 
model = RobertaForSequenceClassification.from_pretrained(path, num_labels=3)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifie

In [5]:
from google.colab import drive
drive.mount('/content/drive')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data set

In [6]:
with open('/content/drive/My Drive/nlp_data/review_dataset/sent_train.json') as json_file:
    train = json.load(json_file)

In [19]:
reviews = train[0]
labels = train[1]

In [20]:
len(labels)

50000

In [21]:
tks = tokenizer(reviews, return_tensors="pt",padding=True, truncation=True,)
input_ids = tks['input_ids']
attention_masks = tks['attention_mask']

In [22]:
input_ids.shape

torch.Size([50000, 512])

In [23]:
# Convert the lists into tensors.
# input_ids = torch.cat(input_ids, dim=0)
# attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
# Print sentence 0, now as a list of IDs.
print('Original: ', reviews[1])
print('Token IDs:', input_ids[1])

Original:  This is the best place at PDX to get breakfast. I love their quiches and their pastries. They are placed before security, so make sure you have enough time to finish all your drinks!
Token IDs: tensor([    0,   713,    16,     5,   275,   317,    23, 11707,  1000,     7,
          120,  7080,     4,    38,   657,    49,  2677,   636,  5065,     8,
           49,   375,  4458,     4,   252,    32,  2325,   137,   573,     6,
           98,   146,   686,    47,    33,   615,    86,     7,  2073,    70,
          110,  6696,   328,     2,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1, 

In [24]:
labels.shape[0]
a = torch.zeros(labels.shape[0], 3)
idx = 0
for x in a:
  x[labels[idx].long()-1] = 1.0
  idx +=1


In [25]:
labels = a
labels

tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        ...,
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]])

## Create Train/Test/Valid splits

In [26]:
total = len(reviews)

num_train = int(total * .6)
num_val = int(total * .3)
num_test = total - num_train - num_val

# make lists of 3-tuples (already shuffled the dataframe in cell above)

train_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train)]
val_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train, num_val+num_train)]
test_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_val + num_train, total)]

train_text = [reviews[i] for i in range(num_train)]
val_text = [reviews[i] for i in range(num_train, num_val+num_train)]
test_text = [reviews[i] for i in range(num_val + num_train, total)]


In [27]:
with torch.no_grad():
  torch.cuda.empty_cache()
torch.cuda.memory_allocated()

9969705472

## Training

In [28]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

batch_size = 2
optimizer = AdamW(model.parameters(),
                  lr = 1e-6, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
epochs = 3

model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [29]:
import numpy as np
# function to get validation accuracy
def get_validation_performance(val_set):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables
    total_correct = 0
    correct_ = np.array([])
    # total_eval_accuracy = 0
    total_eval_loss = 0

    num_batches = int(len(val_set)/batch_size) + 1
    classes = []
    preds = []
    for i in range(num_batches):
      end_index = min(batch_size * (i+1), len(val_set))

      batch = val_set[i*batch_size:end_index]
      if len(batch) == 0: continue
      else:
        input_id_tensors = torch.stack([data[0] for data in batch])
        input_mask_tensors = torch.stack([data[1] for data in batch])
        label_tensors = torch.stack([data[2] for data in batch])
        
        # Move tensors to the GPU
        b_input_ids = input_id_tensors.to(device)
        b_input_mask = input_mask_tensors.to(device)
        b_labels = label_tensors.to(device)
          
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

          # Forward pass, calculate logit predictions.
          outputs = model(b_input_ids, 
                                  token_type_ids=None, 
                                  attention_mask=b_input_mask,
                                  labels=b_labels)
          loss = outputs.loss
          logits = outputs.logits
              
          # Accumulate the validation loss.
          total_eval_loss += loss.item()
          # Move logits and labels to CPU
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          # Calculate the number of correctly labeled examples in batch
          pred_flat = np.argmax(logits, axis=1).flatten()+1
          # labels_flat = label_ids.flatten()
          true_flat = np.argmax(label_ids, axis=1).flatten() +1
          labels_flat = true_flat
          correct_ = np.append(correct_,pred_flat == labels_flat)
          num_correct = np.sum(pred_flat == labels_flat)
          total_correct += num_correct
          classes += list(labels_flat)
          preds += list(pred_flat)
          # Report the final accuracy for this validation run.
    # cm = confusion_matrix(classes, preds)
    avg_val_accuracy = total_correct / len(val_set)
    print(classes)
    print(preds)
    return avg_val_accuracy, classes, preds, correct_

In [30]:
import random

# training loop

# For each epoch...
for epoch_i in range(0, epochs):
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    num_batches = int(len(train_set)/batch_size) + 1

    for i in range(num_batches):
      end_index = min(batch_size * (i+1), len(train_set))

      batch = train_set[i*batch_size:end_index]
      if len(batch)==0: continue

      input_id_tensors = torch.stack([data[0] for data in batch])
      input_mask_tensors = torch.stack([data[1] for data in batch])
      label_tensors = torch.stack([data[2] for data in batch])

      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      b_labels = label_tensors.to(device)

      # Clear the previously calculated gradient
      model.zero_grad()

      # Perform a forward pass (evaluate the model on this training batch).
      # inputs = {}
      # inputs['input_ids'] = b_input_ids
      # inputs['attention_mask'] = b_input_mask
      # outputs = model(**inputs, labels=b_labels)

      # print(b_input_ids.shape)
      # print(b_input_mask.shape)
      # print(b_labels.shape)
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
      loss = outputs.loss
      logits = outputs.logits

      total_train_loss += loss.item()

      # Perform a backward pass to calculate the gradients.
      loss.backward()
      # Update parameters and take a step using the computed gradient.
      optimizer.step()
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set. Implement this function in the cell above.
    print(f"Total loss: {total_train_loss}")
    val_acc, c, p, _ = get_validation_performance(val_set)
    print(f"Validation accuracy: {val_acc}")
    
print("")
print("Training complete!")



======== Epoch 1 / 3 ========
Training...
Total loss: 1768.2909728527302
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 3, 2, 1, 2, 2, 3, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 3, 1, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 3, 2, 1, 3, 3, 2, 2, 2, 2, 2, 3, 3, 3, 1, 3, 1, 2, 2, 1, 3, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 3, 3, 3, 2, 2, 3, 2, 2, 2, 3, 3, 3, 1, 1, 2, 1, 2, 3, 3, 3, 2, 3, 3, 1, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 3, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 1, 1, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2

In [31]:
val_acc, c, p, _ = get_validation_performance(val_set)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 3, 2, 1, 2, 2, 3, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 3, 1, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 3, 2, 1, 3, 3, 2, 2, 2, 2, 2, 3, 3, 3, 1, 3, 1, 2, 2, 1, 3, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 3, 3, 3, 2, 2, 3, 2, 2, 2, 3, 3, 3, 1, 1, 2, 1, 2, 3, 3, 3, 2, 3, 3, 1, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 3, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 1, 1, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [37]:
from sklearn.metrics import precision_score
print(precision_score(c, p, average=None))

[0.80350877 0.96 0.89705882]


In [38]:
path = '/content/drive/MyDrive/Roberta-yelp-sentiment/'

In [39]:
model.save_pretrained(path)
tokenizer.save_pretrained(path)

('/content/drive/MyDrive/Roberta-yelp-sentiment/tokenizer_config.json',
 '/content/drive/MyDrive/Roberta-yelp-sentiment/special_tokens_map.json',
 '/content/drive/MyDrive/Roberta-yelp-sentiment/vocab.json',
 '/content/drive/MyDrive/Roberta-yelp-sentiment/merges.txt',
 '/content/drive/MyDrive/Roberta-yelp-sentiment/added_tokens.json')